# {The Rising Impact of Data Breaches in the U.S.}📝

![Banner](./assets/banner.jpeg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->
I want to talk about cybersecurity and data privacy, specifically analyzing patterns in data 
breaches in the United States. Every year, millions of people are affected by compromised personal 
data, which can result in identity theft, financial loss, and a decline in faith in businesses and 
technology. The rise of cloud platforms, artificial intelligence, and digitalservices has led to a 
rise in the collection of sensitive data, but laws and safeguards are frequently lagging behind. 
This makes the topic important today. Analyzing breach trends can help us identify weak points and 
improve data security for both individuals and corporations.

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->
1. How have data breaches in the US changed over time in terms of both quantity and severity?
2. Which sectors are most commonly the focus of data breaches caused by cyberattacks?
3. What types of data (financial, healthcare, personal identifiers, etc.) are most often compromised, and what patterns exist?

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->
A line chart showing the number of breaches per year.
A bar chart comparing breaches across industries.
A stacked bar or pie chart showing the breakdown of data types exposed.

## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->
1.Annual reports with comprehensive breach statistics are available as files or PDFs from the Identity Theft Resource Center (ITRC).
2.Chronology of Data Breaches by Privacy Rights Clearinghouse is a searchable database that includes variables like date, organization type, and type of data exposed (database/CSV).
3.The Breach Portal of the U.S. Department of Health and Human Services (HHS) contains comprehensive records of data breaches pertaining to healthcare (API and downloadable data).

## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->

In [ ]:
# Start your code here
# Compare breaches by industry
breaches_by_industry = df['Industry'].value_counts().head(10)

plt.figure(figsize=(10,6))
breaches_by_industry.plot(kind='bar')
plt.title("Top 10 Industries by Data Breaches")
plt.xlabel("Industry")
plt.ylabel("Number of Breaches")
plt.show()

## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->
https://www.nist.gov/cyberframework
https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf
https://privacyrights.org/
https://www.idtheftcenter.org/

In [ ]:
# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python source.ipynb



[NbConvertApp] Converting notebook source.ipynb to python
[NbConvertApp] Writing 1271 bytes to source.py
